#  Evaluation & Comparison of Recommender Systems
This notebook covers the evaluation of collaborative filtering models,
comparison with content-based recommendations, and highlights key insights.


In [1]:
from google.colab import drive
drive.mount('/content/drive')
print("Drive mounted succesfully")

Mounted at /content/drive
Drive mounted succesfully


In [5]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Recommender system tools
from sklearn.metrics import mean_squared_error
print("Libraris imported successfully")

Libraris imported successfully


In [6]:
import os

# Project folder path
project_path = "/content/drive/MyDrive/Projects/Machine_Learning/Course_Recommendation_System"
data_path = os.path.join(project_path, "data", "raw")

# Check folder contents
print("📂 Checking data folder contents...")
print(os.listdir(data_path))

📂 Checking data folder contents...
['udemy_course_data.csv']


In [7]:
import pandas as pd

# Raw file
raw_file = os.path.join(data_path, "udemy_course_data.csv")

# Load dataset
df = pd.read_csv(raw_file)

# Confirmation
print("\n✅ Dataset loaded successfully!")
print("📊 Shape of data:", df.shape)
df.head()



✅ Dataset loaded successfully!
📊 Shape of data: (3683, 18)


,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject,profit,published_date,published_time,year,month,day
0,1070968,Ultimate Investment Banking Course,https://www.udemy.com/ultimate-investment-bank...,True,200,2147,23,51,All Levels,1.5 hours,2017-01-18T20:58:58Z,Business Finance,429400,2017-01-18,20:58:58Z,2017,1,18
1,1113822,Complete GST Course & Certification - Grow You...,https://www.udemy.com/goods-and-services-tax/,True,75,2792,923,274,All Levels,39 hours,2017-03-09T16:34:20Z,Business Finance,209400,2017-03-09,16:34:20Z,2017,3,9
2,1006314,Financial Modeling for Business Analysts and C...,https://www.udemy.com/financial-modeling-for-b...,True,45,2174,74,51,Intermediate Level,2.5 hours,2016-12-19T19:26:30Z,Business Finance,97830,2016-12-19,19:26:30Z,2016,12,19
3,1210588,Beginner to Pro - Financial Analysis in Excel ...,https://www.udemy.com/complete-excel-finance-c...,True,95,2451,11,36,All Levels,3 hours,2017-05-30T20:07:24Z,Business Finance,232845,2017-05-30,20:07:24Z,2017,5,30
4,1011058,How To Maximize Your Profits Trading Options,https://www.udemy.com/how-to-maximize-your-pro...,True,200,1276,45,26,Intermediate Level,2 hours,2016-12-13T14:57:18Z,Business Finance,255200,2016-12-13,14:57:18Z,2016,12,13


In [9]:
# Example: create dummy user-item interactions
import numpy as np

# Randomly create 50 users × 100 courses matrix (for demo/testing)
num_users = 50
num_courses = 100

# Select first 100 courses from dataset
courses_subset = df['course_id'].iloc[:num_courses]

# Random interactions (0 = not enrolled, 1 = enrolled)
user_item_matrix = pd.DataFrame(
    np.random.randint(0, 2, size=(num_users, num_courses)),
    columns=courses_subset
)

print("✅ Dummy user-item matrix created")
user_item_matrix.head()

✅ Dummy user-item matrix created


course_id,1070968,1113822,1006314,1210588,1011058,192870,739964,403100,476268,1167710,...,891484,1217064,382204,1259560,308696,1270254,474928,1148774,959144,1233350
0,1,0,0,1,1,0,0,0,1,1,...,1,0,0,1,0,1,0,1,0,0
1,0,1,1,1,0,1,0,0,1,0,...,0,1,1,0,1,1,0,0,1,1
2,1,1,1,0,1,1,0,1,1,1,...,0,0,0,1,0,1,0,1,0,0
3,1,1,0,1,0,1,0,1,1,1,...,0,0,1,0,0,1,1,1,1,0
4,1,1,0,1,1,0,1,1,1,0,...,0,0,0,1,0,1,1,0,0,0


In [15]:
# =========================
# 10. Collaborative Filtering (Neural Network Embedding)
# =========================

# 🔹 Step 1: Prepare training data
# Assuming user_item_matrix is already loaded
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dot, Flatten
from tensorflow.keras.optimizers import Adam

# Create user and course indices
user_ids = np.array(user_item_matrix.index)
course_ids = np.array(user_item_matrix.columns)

# Convert user-item matrix to list of triplets: (user, course, rating)
user_idx, course_idx = np.where(~np.isnan(user_item_matrix.values))
ratings = user_item_matrix.values[user_idx, course_idx]

train_user = user_idx
train_course = course_idx
train_ratings = ratings

# 🔹 Step 2: Build NN Embedding model
n_users = len(user_ids)
n_courses = len(course_ids)
embedding_size = 50  # You can adjust

user_input = Input(shape=(1,))
user_embedding = Embedding(n_users, embedding_size, input_length=1)(user_input)
user_vec = Flatten()(user_embedding)

course_input = Input(shape=(1,))
course_embedding = Embedding(n_courses, embedding_size, input_length=1)(course_input)
course_vec = Flatten()(course_embedding)

dot = Dot(axes=1)([user_vec, course_vec])

nn_model = Model([user_input, course_input], dot)
nn_model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['accuracy'])

# 🔹 Step 3: Train model
nn_model.fit([train_user, train_course], train_ratings, epochs=5, batch_size=32, verbose=1)

# 🔹 Step 4: Save trained model for recommendations
nn_model_trained = nn_model
print("✅ Neural Network Embedding model training done!")

# 🔹 Step 5: Recommendation function
def recommend_courses_nn(user_id, user_item_matrix, model, top_n=5):
    if user_id not in user_item_matrix.index:
        print("❌ User ID not found in matrix.")
        return

    user_idx = user_item_matrix.index.get_loc(user_id)
    course_indices = np.arange(len(user_item_matrix.columns))

    predictions = model.predict([np.full_like(course_indices, user_idx), course_indices], verbose=0).flatten()

    top_indices = predictions.argsort()[::-1][:top_n]
    top_courses = user_item_matrix.columns[top_indices]

    print(f"\n Top {top_n} recommended courses for User {user_id}:")
    for i, course_id in enumerate(top_courses):
        print(f"{i+1}. {course_id}")

# 🔹 Step 6: Example: Recommend for user_id = 0
recommend_courses_nn(user_id=0, user_item_matrix=user_item_matrix, model=nn_model_trained)

Epoch 1/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.5015 - loss: 0.4982
Epoch 2/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4995 - loss: 0.4524
Epoch 3/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5130 - loss: 0.2690
Epoch 4/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6050 - loss: 0.2358
Epoch 5/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6312 - loss: 0.2297
✅ Neural Network Embedding model training done!

 Top 5 recommended courses for User 0:
1. 1210588
2. 285638
3. 43319
4. 606928
5. 302562
